# **EcoSort**
EcoSort es un proyecto open source creado durante el Hackathon 2023 como una herramienta que podría ayudar a combatir el problema de la contaminación. El principal objetivo de EcoSort es el poder ayudar a la población en general en la correcta segmentación de sus desperdicios.

Para realizar esta tarea se optó por entrenar una red neuronal en Tensorflow que tenga como entrada la imagen del objeto y a su salida la clasificación en donde debe ser colocada.

*Uno de los objetivos del proyecto es que la efectividad del modelo sea mayor a un 90%*

In [13]:
# Importación de librerías necesarias
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
# Set the seed
tf.random.set_seed(42)

## Obtención de datos de entrenamiento

Para el entrenamiento de este modelo se ha encontrado el dataset ["Garbage Classification](https://www.kaggle.com/datasets/asdasdasasdas/garbage-classification).

Una vez obtenidos los datos se realizará el preprocesamiento de los mismos, lo cual consistirá de normalizar las imagen y reescalarlas todas a un tamaño de (224,224); Además de esto se aplicará la técnica de "Data Augmentation" para poder generar mayor cantidad de datos de entrenamiento para la red.

In [ ]:
!pip install unrar
!unrar x "test.rar"
!unrar x "train.rar"

In [15]:
# Preproces the data, normalize it and apply data augmentation
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 20,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

test_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 20,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2)

In [5]:
# Setup dir of the data
train_dir = "/content/train"
test_dir = "/content/test"

# Import data from directories and batch it
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224,224),
                                               class_mode="categorical",
                                               seed=42)

test_data = test_datagen.flow_from_directory(test_dir,
                                             batch_size=32,
                                             target_size=(224,224),
                                             class_mode="categorical",
                                             seed=42)


Found 2025 images belonging to 6 classes.
Found 502 images belonging to 6 classes.


## Creación del modelo

In [8]:
# Importar las capas que se utilizarán
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
import tensorflow_hub as hub

In [33]:
# Creación del primer modelo utilizando transfer-learning 
model_url= "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

feature_extraction_layer = hub.KerasLayer(model_url,
                                          trainable=False,
                                          name="feature_extraction_layer",
                                          input_shape=(224,224,3))

model_1 = Sequential([
    feature_extraction_layer,
    Dense(6, activation="softmax")
])

# Compilar el modelo
model_1.compile(loss="categorical_crossentropy",
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                 metrics=["accuracy"])

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

# Entrenar el modelo
history_1 = model_1.fit(train_data,
                         epochs=20,
                         steps_per_epoch=len(train_data),
                         validation_data=test_data,
                         validation_steps=len(test_data),
                        callbacks=[early_stopping_callback])

Epoch 1/20
64/64 [==============================] - 48s 603ms/step - loss: 0.5952 - accuracy: 0.7911 - val_loss: 0.7665 - val_accuracy: 0.8028
Epoch 2/20
64/64 [==============================] - 37s 584ms/step - loss: 0.2844 - accuracy: 0.8978 - val_loss: 0.7780 - val_accuracy: 0.8048
Epoch 3/20
64/64 [==============================] - 37s 579ms/step - loss: 0.2278 - accuracy: 0.9121 - val_loss: 0.7509 - val_accuracy: 0.7948
Epoch 4/20
64/64 [==============================] - 37s 587ms/step - loss: 0.1787 - accuracy: 0.9314 - val_loss: 0.7623 - val_accuracy: 0.8267
Epoch 5/20
64/64 [==============================] - 37s 584ms/step - loss: 0.1468 - accuracy: 0.9467 - val_loss: 0.7051 - val_accuracy: 0.8267
Epoch 6/20
64/64 [==============================] - 37s 579ms/step - loss: 0.1155 - accuracy: 0.9625 - val_loss: 0.8070 - val_accuracy: 0.8327
Epoch 7/20
64/64 [==============================] - 37s 583ms/step - loss: 0.0966 - accuracy: 0.9689 - val_loss: 0.7629 - val_accuracy: 0.8147

In [34]:
model_1.save("modelo_85.h5")

In [36]:
model_1.save_weights("Weights/model_85_weights.cpkt")

In [38]:
import cv2

In [61]:
img = cv2.imread("/content/test/paper/paper534.jpg")
img_res = cv2.resize(img, (224,224))
img_res = img_res/255.

prediction = model_1.predict(tf.expand_dims(img_res, axis=0))
prediction = tf.squeeze(prediction)

max_index = tf.argmax(prediction)
max_index.numpy()

1/1 [==============================] - 0s 47ms/step


3

In [ ]:
def prepare_image(img_path):
  img = cv2.imread(img_path)
  img_res = cv2.resize(img, (224,224))
  img_res = img_res/255.0

  return tf.expand_dims(img_res, axis=0)

In [64]:
from tensorflow import keras

modelo2 = keras.models.load_model("/content/modelo_85.h5",
                                  custom_objects={'KerasLayer':hub.KerasLayer})